In [31]:
"""
This code makes use of the code written by Jimmy Song in the book Programming Bitcoin.
This code only tries to build on Jimmi Song's code and take it to the next step as 
suggested by him in chapter 14 of his book.
This module is coded entirely by Oscar Serna.

This is just an educational-purpose code. The author does not take any responsibility
on any losses caused by the use of this code.
"""

from ecc import PrivateKey, S256Point, Signature
from script import p2pkh_script, p2sh_script, Script, p2wpkh_script,p2wsh_script
from helper import (
    decode_base58, SIGHASH_ALL, h160_to_p2pkh_address, hash160, 
    h160_to_p2sh_address, encode_varint)
from tx import TxIn, TxOut, Tx, TxFetcher
import hashlib
from bip32 import Xtended_privkey, Xtended_pubkey
from bip39 import Mnemonic
import segwit_addr

class MasterAccount(Xtended_privkey):
    
    @classmethod
    def generate_random(self,entropy = 128,  passphrase="", testnet = False):
        mnemonic = Mnemonic.generate_random(entropy, passphrase)
        print(f"Copy these words for future recovery:\n{mnemonic.words}")
        return self.from_bip39_seed(int.from_bytes(mnemonic.seed,"big"), testnet = testnet)
        
    @classmethod
    def recover_from_words(self,mnemonic_list = None, entropy = 128,  passphrase="", testnet = False):
        if isinstance(mnemonic_list,str):
            mnemonic_list = mnemonic_list.split()
        mnemonic = Mnemonic.recover_from_words(mnemonic_list, entropy ,  passphrase)
        return self.from_bip39_seed(int.from_bytes(mnemonic.seed,"big"), testnet = testnet)
    
        

class Account():
    
    """
    To create account use from_phrase() method.
    """
    
    def __init__(self, _privkey,addr_type = "P2PKH",testnet=False):
        """
        Initialize the account with a private key in Integer form.
        addr_type = String. Possible values: "P2PKH","P2WPKH","P2SH_P2WPKH"
        testnet: Boolean. If Testnet network is desired, simply specify this value True.
        Otherwise, simply ommit it.
        
        """
        self.privkey = PrivateKey(_privkey)
        self.addr_type = addr_type.lower()
        self.testnet = testnet
        self.redeem_script_segwit = p2wpkh_script(self.privkey.point.hash160())
        serialized_redeem = self.redeem_script_segwit.raw_serialize()
        
        if self.addr_type == "p2pkh":
            self.address = self.privkey.point.address(testnet = testnet)
        elif self.addr_type == "p2wpkh":
            if self.testnet:
                self.address = segwit_addr.encode("tb",0,self.privkey.point.hash160())
            else:
                self.address = segwit_addr.encode("bc",0,self.privkey.point.hash160())
        elif self.addr_type == "p2sh_p2wpkh":
            self.address = h160_to_p2sh_address(hash160(serialized_redeem), testnet=testnet)
        else:
            raise Exception("Addres type not supported. Must be p2pkh, p2wpkh, or p2sh_p2wpkh")
        
        
        
    def __repr__(self):
        return f"Private Key Hex: {self.privkey.hex()}"
        
    
    @classmethod
    def from_phrase(cls,phrase, endian="big",addr_type = "P2PKH",testnet=False):
        """
        phrase must be in bytes.
        endian can be either "big" or "little". Change later to boolean "bignendian"
        Returns an account using the phrase chosen which is converted to an Integer
        """
        if endian not in ["big","little"]:
            raise Exception( f'endian can be either "big" or "little" not "{endian}"')
            
        if isinstance(phrase, str):
            phrase = phrase.encode('utf-8')
            
        if isinstance(phrase, bytes):
            return cls(int.from_bytes(phrase,endian),addr_type,testnet)
        
        else:
            raise Exception( f"The phrase must be a string or bytes, not {type(phrase)}" )


class MultSigAccount():
    
    def __init__(self,m, n, _privkey, public_key_list, addr_type="p2sh", testnet = False, segwit=True):
        """
        Initialize the account with a private key in Integer form.
        m: the minumin amount of signatures required to spend the money.
        n: total amount of signatures that can be used to sign transactions.
        Note: Therefore:
        a- m has to be less or equal to n.
        b- n has to be equal to the length of the array _privkeys
        Also:
        c- n could be 20 or less, but to keep simplicity in the code, n can only be 16 or less.
        private_key must be an int.
        addr_type = String. Possible values: "P2SH","P2WSH","P2SH_P2WSH". NOT case sensitive.
        Public keys must be in bytes
        """
        if n != len(public_key_list):
            raise Exception("n must be equal to the amount of public keys")
        if m < 1 or m > 16 or n < 1 or n > 16:
            raise Exception("m and n must be between 1 and 16")
        if m > n:
            raise Exception("m must be always less or equal than n")
            
        index = -1
        pubkey = PrivateKey(_privkey).point.sec()
        for i,public_key in enumerate(public_key_list):
            if public_key == pubkey:
                index = i
        if index < 0: raise Exception ("Private key must correspond to one of the public keys.")
    
        
        self.privkey = PrivateKey(_privkey)
        self.public_keys = public_key_list
        self.m = m
        self.n = n
        self.testnet = testnet
        self.address = h160_to_p2sh_address(hash160(serialized_redeem), testnet=self.testnet)
        self.addr_type = addr_type.lower()
        self.privkey_index = index
        self.redeem_script = Script([m+80, *self.public_keys, n + 80, 174])
        serialized_redeem = self.redeem_script.raw_serialize()
      
        if self.addr_type == "p2sh":
            self.address = h160_to_p2sh_address(hash160(serialized_redeem), testnet=self.testnet)
        elif self.addr_type == "p2wsh":
            if self.testnet:
                self.address = segwit_addr.encode("tb",0,hashlib.sha256(serialized_redeem).digest())
            else:
                self.address = segwit_addr.encode("bc",0,hashlib.sha256(serialized_redeem).digest())
        elif self.addr_type == "p2sh_p2wsh":
            address_script = p2wsh_script(hashlib.sha256(serialized_redeem).digest())
            serialized_addr_script = address_script.raw_serialize()
            self.address = h160_to_p2sh_address(hash160(serialized_addr_script), testnet=testnet)
        else:
            raise Exception("Addres type not supported. Must be p2sh, p2wsh, or p2sh_p2wsh")
        
        
    def __repr__(self):
        return f"Private Key Hex: {self.privkey.hex()}"
        
    
    @classmethod
    def from_phrases(cls, m , n , phrases, testnet = False):
        """
        phrase: must be a list of tuples of (bytes, endian) or (string, endian). i.e:
            [(b"my secret","little"),("my other secret","big")]
            
        endian: can be either "big" or "little"
        Returns a multisignature account using the phrases chosen which are converted to Integers
        """
        keys = [int.from_bytes(key[0],key[1]) for key in phrases]
        return cls(m,n,keys,testnet)
       
 

In [2]:
       
class Transact:
    
    @classmethod
    def get_index(self,outs_list, address):    
        """
        For later: implement looking for multiple indexes in the same tx.
        """
       
        print(f"Address: {address}")
        if address[:2] in ["tb","bc"]:
            address = self.decode_p2wpkh_addr(address)
            for index,out in enumerate(outs_list):
                if out.script_pubkey.cmds[1] == address or out.script_pubkey.cmds[0] == address:
                    return index
        else:
            address = decode_base58(address)
            for index,out in enumerate(outs_list):
                if out.script_pubkey.cmds[2] == address or out.script_pubkey.cmds[1] == address:
                    return index
        
        
        raise Exception( "output index not found")

    @classmethod
    def get_amount_utxo(cls,outs_list, index):
        """
        Supporting method.
        receives the list of outputs from transaction and the index of 
        particular output of interest and returns the amount of the UTXO.
        outs_list: is the list of outputs of previous transaction where the UTXO is.
        index: the index of the UTXO in the list of all the outputs.
        """
        return outs_list[index].amount

    @classmethod
    def get_tx_ins_utxo(self,prev_tx_id_list, receiving_address, testnet=True):
        """
        Receives a list of transaction ids where the UTXOs to spend are, and 
        also the receiving address to return a valid tx_in list to create
        a transaction.
        prev_tx_id_list: list of the transaction ids where the UTXOs are.
        receiving_address: the address trying to spend the UTXO (String).
        testnet: if the transaction is in testnet or not (boolean).
        """
        tx_ins = []

        for prev_tx_id in prev_tx_id_list:
            prev_tx = TxFetcher.fetch(prev_tx_id, testnet)
            prev_index = self.get_index(prev_tx.tx_outs, receiving_address)
            tx_in = TxIn(bytes.fromhex(prev_tx_id),prev_index)
            utxo = self.get_amount_utxo(prev_tx.tx_outs, prev_index)
            #print(f"index 1: {prev_index}, amount: {utxo}")
            tx_ins.append({"tx_in": tx_in, "utxo": utxo})

        return tx_ins

    @classmethod
    def calculate_fee(cls,version, tx_ins, tx_outs, locktime, privkey, redeem_script, 
                      testnet=True, multisig =False, fee_per_byte = 8, segwit = False ):
        """
        privkey: can be just one or a list of private keys in the case of multisignature.
        """
        my_tx = Tx(1, tx_ins, tx_outs, 0, testnet=True)
        print(my_tx)

        # sign the inputs in the transaction object using the private key
        if multisig:
            #for tx_input in range(len(tx_ins)):
             #   print(my_tx.sign_input_multisig(tx_input, privkey, redeem_script))
                pass
        else:
            for tx_input in range(len(tx_ins)):
                print(my_tx.sign_input(tx_input, privkey, segwit = segwit))
            # print the transaction's serialization in hex

        #Let's calculate the fee and the change:
        tx_size = len(my_tx.serialize().hex())
        #fee_per_byte = 8 # I changed from 2 to 10 after sending this transaction because it had really low appeal to miners.
        if multisig: tx_size*=2
        fee = tx_size * fee_per_byte
        print(f"fee: {fee}")
        return fee
    
    @classmethod
    def calculate_change(cls, utxo_list, fee, amountTx):
        """
        utxo_list: the list of the amounts of every utxo.
        amountTx: the list of the ammounts of every transaction output.
        fee: the fee of the transaction.
        Returns the respective amount of the change.
        """
        total_utxo = sum(utxo_list)
        total_out = sum(amountTx)
        change = total_utxo - fee - total_out
        print(f"change {change}")
        if change < 0:
            raise Exception( f"Not enough utxos: total_utxo {total_utxo} and total_out {total_out} meaning change = {change}")
        #Let's make sure that we are actually spending the exact amount of the UTXO
        total_send=fee+total_out+change
        diff = total_utxo-total_send
        print(f"total {total_send}, diff: {diff}")

        return change
    
    @classmethod
    def decode_p2wpkh_addr(self,address):
        decoded = segwit_addr.bech32_decode(address)
        str_pubkeyhash = ""
        for char in decoded[1][1:]:
            str_pubkeyhash += "{0:05b}".format(char)
        return int(str_pubkeyhash,2).to_bytes(20,"big")
    
   

In [3]:
class Transaction(Transact):
      
    def __init__(self,transaction,sender_account,tx_ins,utxos ,tx_outs, fee, change, testnet, segwit):
        """
        utxo_tx_id_list: the list of the transaction ids where the UTXOs are.
        receivingAddress_w_amount_list: a list of tuples (to_address,amount) specifying
        the amount to send to each address.
        sender_account: must be an Account object.
        If fee is specifyed, then the custom fee will be applied.
        """
        self.transaction = transaction
        self.sender_account = sender_account
        self.tx_ins = tx_ins
        self.utxos = utxos
        self.fee = fee
        self.segwit = segwit
        self.tx_outs = tx_outs
        self.testnet = testnet
        self.change = change
        
    @classmethod
    def validate_data(self,sender_account_address,outputAddress_amount_list ):
        if sender_account_address[0] in "2mnt":
            testnet = True
            for addr in outputAddress_amount_list:
                if addr[0][0] not in "2mnt":
                    raise Exception (f"{addr[0]} not a testnet address. Funds will be lost!")
                if addr[1] < 1:
                    raise Exception (f"{addr[1]} not a valid amount. It should be greater than 1 satoshis")
        else:
            testnet  = False
            for addr in outputAddress_amount_list:
                if addr[0][0] not in "13b":
                    raise Exception (f"{addr[0]} not a mainnet bitcoin address. Funds will be lost!")
                if addr[1] < 1:
                    raise Exception (f"{addr[1]} not a valid amount. It should be greater than 1 satoshis")
              
        return testnet
    
    @classmethod
    def get_outputs(self, receivingAddress_w_amount_list, account):
        
        #https://en.bitcoin.it/wiki/List_of_address_prefixes
        #We create the tx outputs based on the kind of address
        tx_outs = []
        for output in receivingAddress_w_amount_list:
            
            if output[0][0] in "1mn" :
                tx_outs.append( TxOut(output[1], p2pkh_script(decode_base58(output[0]))))
            #For multidignature p2sh
            elif output[0][0] in "23" :
                tx_outs.append( TxOut(output[1], p2sh_script(decode_base58(output[0]))))
            elif output[0][:2] in ["bc","tb"] :
                tx_outs.append( TxOut(output[1], p2wpkh_script(self.decode_p2wpkh_addr(output[0]))))#Needs to be tested
            else:
                raise Exception ("Not supported address.")
        
        #Let's return the fake change to our same address. 
        #This will change later when BIP32 is implemented.
        if account.addr_type == "p2pkh":
            tx_outs.append( TxOut(1000000, p2pkh_script(decode_base58(account.address))))
        elif account.addr_type == "p2wpkh":
            tx_outs.append( TxOut(1000000, p2wpkh_script(self.decode_p2wpkh_addr(account.address))))
        elif account.addr_type in  ["p2sh","p2sh_p2wpkh"]:
            tx_outs.append( TxOut(1000000, p2sh_script(decode_base58(account.address))))
        
        return tx_outs
                
    @classmethod
    def get_inputs(self,utxo_tx_id_list, sender_address, testnet):
        return self.get_tx_ins_utxo(utxo_tx_id_list, sender_address, testnet)
         
    
    @classmethod
    def unsigned_tx(self,tx_ins, tx_outs, testnet, segwit, sender_account, utxos,receivingAddress_w_amount_list):
        #CREATING THE TRANSACTION RIGHT HERE:
        my_tx = Tx(1, tx_ins, tx_outs, 0, testnet=testnet, segwit=segwit)#check for segwit later!!!!
        
        #CHECK THE FOLLOWING LINE LATER!! 
        if sender_account.addr_type not in  ["p2sh","p2wsh","p2sh_p2wsh"]:
            fee = self.calculate_fee(1, tx_ins, tx_outs, 0, privkey=sender_account.privkey, 
                                         redeem_script=None, testnet=testnet, segwit = segwit)
            
            change = self.calculate_change(utxos, fee, [x[1] for x in receivingAddress_w_amount_list])

        else: raise Exception ("This is a multisignature transaction. Use MultiSigTransaction instead of Transaction.")
        
        my_tx.tx_outs[-1].amount = change
        return fee, change, my_tx
        
        
    @classmethod
    def sign_tx(self, tx_ins, transaction, sender_account, segwit):
        p2sh = False
        if sender_account.addr_type in ["p2sh","p2sh_p2wpkh"]: p2sh = True
        for tx_input in range(len(tx_ins)):
            if not transaction.sign_input(tx_input, sender_account.privkey, segwit = segwit, p2sh = p2sh):
                return False
        return True
    
    @classmethod
    def create(self, utxo_tx_id_list, receivingAddress_w_amount_list, sender_account, fee=None, #Modify code to allow manual fee!!!
                 segwit=False):
        """
        utxo_tx_id_list: the list of the transaction ids where the UTXOs are.
        receivingAddress_w_amount_list: a list of tuples (to_address,amount) specifying
        the amount to send to each address.
        sender_account: must be an Account object.
        If fee is specifyed, then the custom fee will be applied.
        """
        testnet = self.validate_data(sender_account.address,receivingAddress_w_amount_list)
        tx_outs = self.get_outputs(receivingAddress_w_amount_list, sender_account)
        tx_ins_utxo = self.get_inputs(utxo_tx_id_list, sender_account.address, testnet)
        tx_ins = [x["tx_in"] for x in tx_ins_utxo]
        utxos = [x["utxo"] for x in tx_ins_utxo]
        fee, change, transaction = self.unsigned_tx(tx_ins, tx_outs, testnet, segwit, sender_account, utxos,receivingAddress_w_amount_list)
        if self.sign_tx( tx_ins, transaction, sender_account, segwit):
            return Transaction(transaction, sender_account, tx_ins,utxos ,tx_outs, fee, change, testnet, segwit)
        else:
            raise Exception("Signature faliled") 
            

In [4]:
class MultiSigTransaction(Transaction):
    
    @classmethod
    def unsigned_tx(self,tx_ins, tx_outs, testnet, segwit, sender_account, utxos,receivingAddress_w_amount_list):
        
        my_tx = Tx(1, tx_ins, tx_outs, 0, testnet=testnet, segwit=segwit)
        
        fee = self.calculate_fee(1, tx_ins, tx_outs, 0, privkey=sender_account.privkey,
                                redeem_script=sender_account.redeem_script,testnet=testnet, 
                                segwit = segwit, multisig =True)
        
        change = self.calculate_change(utxos, fee, [x[1] for x in receivingAddress_w_amount_list])
        
        my_tx.tx_outs[-1].amount = change
        return fee, change, my_tx

    @classmethod
    def sign1by1(self, transaction, sender_account):
       
        for tx_input in range(len(transaction.tx_ins)):
            transaction.sign_input_multisig_1by1(tx_input, sender_account.privkey,sender_account.privkey_index,
                                                 sender_account.redeem_script, sender_account.n)
        return transaction
        
    @classmethod
    def verify_signatures(self, transaction, sender_account):
        if transaction.verify_signatures( sender_account.m):
            return transaction
        else:
            return False
    
    @classmethod
    def create(self, utxo_tx_id_list, receivingAddress_w_amount_list, sender_account, fee=None, #Modify code to allow manual fee!!!
                 segwit=False):
        """
        utxo_tx_id_list: the list of the transaction ids where the UTXOs are.
        receivingAddress_w_amount_list: a list of tuples (to_address,amount) specifying
        the amount to send to each address.
        sender_account: must be an Account object.
        If fee is specifyed, then the custom fee will be applied.
        """
        testnet = self.validate_data(sender_account.address,receivingAddress_w_amount_list)
        tx_outs = self.get_outputs(receivingAddress_w_amount_list, sender_account)
        tx_ins_utxo = self.get_inputs(utxo_tx_id_list, sender_account.address, testnet)
        tx_ins = [x["tx_in"] for x in tx_ins_utxo]
        utxos = [x["utxo"] for x in tx_ins_utxo]
        fee, change, transaction = self.unsigned_tx(tx_ins, tx_outs, testnet, segwit, sender_account, utxos,receivingAddress_w_amount_list)
        transaction = self.sign1by1(transaction, sender_account)
        final_tx =  self.verify_signatures(transaction, sender_account)
        if isinstance(final_tx,bool):
            return MultiSigTransaction(transaction, sender_account, tx_ins,utxos ,tx_outs, fee, change, testnet, segwit)
        else:
            return final_tx
        
    @classmethod
    def sign_tx():
        raise Exception ("Unsupported for multisignature transactions. Use sign1by1() method instead." )
    
    

In [5]:
account1 = Account.from_phrase(b"Oscar Eduardo Serna Rosero","big","p2pkh",True)

accountP2wPKH = Account.from_phrase(b"Oscar Eduardo Serna Rosero","big","p2wpkh",True)

segwitAccount = Account.from_phrase(b"Oscar Eduardo Serna Rosero","big","p2sh_p2wpkh",True)

print(accountP2wPKH.address)

prev_tx_id_list = ["5e3676a90605efaaef3aae83be71b27f7aa4a2f2c5c39695b4e06b4cbb555a09"]
my_tx = Transaction.create(prev_tx_id_list, [(segwitAccount.address, 24000)], accountP2wPKH, segwit = True)
#my_tx.create()

tb1q22gralqsqn0qrzpm5d58hc4gaf8kkxcer6tcld
RAW RESPONSE: b'\x02\x00\x00\x00\x00\x01\x01\xea\xfb\xaf\x95:e\xec\x0b\xd1&A`P\x98#.>m\x93\x8a\xdc\xb6<\xdc\xba;-y\x84\xca4j\x01\x00\x00\x00\x17\x16\x00\x14\xf4\x18\x8c\x05\xf6\x12\x8a\xaaaY+\x0f\x1fG]\x98\x18v2&\xfe\xff\xff\xff\x02P\xc3\x00\x00\x00\x00\x00\x00\x16\x00\x14R\x90>\xfc\x10\x04\xde\x01\x88;\xa3h{\xe2\xa8\xeaOk\x1b\x19YkE\x00\x00\x00\x00\x00\x16\x00\x14\x06\x06\xaaaka\xdb\xc1\x90A9~\xdb\xca\x8d^\xd2\xf2&\xbc\x02G0D\x02 \x17`\xc5\xeb\xac\xb7*a\xaa\xa4\xb7\x8d\x8e\x08\xb0\xc4\x1d\r\x13\xf4\xb9\x84\x1a*\xb5\xf1O\xe0\n\x9f\x86u\x02 o\x1d\xe8rE\xa2\xe6w\xf3\x8bB\xfc$kcr_\xefpFm\xb7\xa6Jt\x18\xde\n\xbaw\x9c\xa3\x01!\x03\xbc7mh\x13\xfa\x98\xd6v\xb9\xdc\xd1\x1dcB\x9d\x9b]\x86\xa7)\xc0$\xd2\xbc\xa2\xacQ!\x82\x9c\xcc\x8e\xd4\x19\x00'
FIRST CASE: 0
length script: 23
current_byte : 22
length script: 22
current_byte : 0
op_code : 0
current_byte : 20
length script: 22
current_byte : 0
op_code : 0
current_byte : 20
6a34ca84792d3bbadc3cb6dc8a936d3

In [6]:
my_tx.transaction.serialize().hex()

'01000000000101095a55bb4c6be0b49596c3c5f2a2a47a7fb271be83ae3aefaaef0506a976365e0000000000ffffffff02c05d00000000000017a914f7c07f67fb6e54ea6264b1a508b24e34ec999c5687705e00000000000016001452903efc1004de01883ba3687be2a8ea4f6b1b1902473044022005b8d205f23f3a6fd2cc57be5495b0aa13c591a8903eb0a541a484d4efaeb0ac0220367b1384bd2d88e4fb4ffad55e68b0f8a144b7fafcf1347abc669d5e8804edfa012103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e400000000'

In [7]:
my_tx.sender_account

Private Key Hex: 0000000000004f73636172204564756172646f205365726e6120526f7365726f

In [8]:
private_key = int.from_bytes(b"Oscar Eduardo Serna Rosero","big")
public_keys = [ PrivateKey(private_key).point.sec() , PrivateKey(int.from_bytes(b"Oscar Eduardo Serna Rosero","little")).point.sec()]
multisig_acc = MultSigAccount(2,2,private_key, public_keys,"p2sh",True)

In [9]:
private_key = int.from_bytes(b"Oscar Eduardo Serna Rosero","little")
public_keys = [PrivateKey(int.from_bytes(b"Oscar Eduardo Serna Rosero","big")).point.sec(), PrivateKey(private_key).point.sec() ]
multisig_acc2 = MultSigAccount(2,2,private_key, public_keys,"p2sh",True)

In [10]:
multisig_acc.privkey_index

0

In [11]:
multisig_acc2.privkey_index

1

In [12]:
multisig_acc2.address

'2NAqp3V17G4iBekEwWuYkgzoEyWiULhYZLs'

In [13]:
multisig_acc.address

'2NAqp3V17G4iBekEwWuYkgzoEyWiULhYZLs'

In [14]:
account1.address

'mo3WWB4PoSHrudEBik1nUqfn1uZEPNYEc8'

In [15]:
prev_tx_id_list = ["032c28ea3b77768577c54847e155d482e47dfa7a835b2394e5a3c4427f5d2fcc"]
my_tx = MultiSigTransaction.create(prev_tx_id_list, [(account1.address,200000)], multisig_acc, fee=None, #Modify code to allow manual fee!!!
                 segwit=False)

RAW RESPONSE: b"\x01\x00\x00\x00\x01\xea\xa1\xef\x94\xafx\x90FE\xf7\xd5\x96\xe1\x16a\xac*V\x1a\xc5\xc7\x99\x9bzz#\x97\xe4*<\xc1\xf3\x00\x00\x00\x00\xda\x00H0E\x02!\x00\xb4r\xf9\x87\xa5\x811\x0bNkK\xbd\xff\xe5\x9cZ\x9adBV\xfe\xbf@.j\xea%\xc6\x8f|\xc2W\x02 ~\xfc\xde\xbb=;e\xe0`d%\x12B\x83&Z\xdf\xc3Y\xef\x90\xf9\xd2{5\x87\xd0\xc1\x80\x16Mr\x01G0D\x02 L\x85\x8b!\xf2\x01c\xd9&pR\xfa\xd5\x89\xe4\x08,\x98\n:~\xf0+\xbe\xe9b\x16@9\xcdE\xa7\x02 \x19\x04o\neJ\x16\xa0\xabJYX\xb9\xfd\xd8T@~\x08\xfa\x87>\x16\xaf\xda\x06\xf4bJ\x94\xd8\xf6\x01GR!\x03\xe0\x7f\x96\xe5\xbaY\x841\xc0\xc9\x94I:J\xe9\x88\xc9\x85L\x17\x1d]K\xb1@\xdb\n'\xa4\xc8S\xe4!\x03\x1bc\xf9d\xd8\xc6]\x1d\x116\xdc\xfeP3\xde\xde\xa8\x8c-A\x194\xeaH\xc9p\x84\x10\xbe\x84\xe5\xeeR\xae\xff\xff\xff\xff\x02 \xa1\x07\x00\x00\x00\x00\x00\x17\xa9\x14\xf7\xc0\x7fg\xfbnT\xeabd\xb1\xa5\x08\xb2N4\xec\x99\x9cV\x87@\x8c\x07\x00\x00\x00\x00\x00\x17\xa9\x14\xc1\x04\xb5v\xf5Cc\tXz\xef\xa3\xdd\xdd\xd5\xc2\x95\xb9\x04\x80\x87\x00\x00\x00\x00"
SECOND CASE: 1


In [16]:
my_tx_signed = MultiSigTransaction.sign1by1(my_tx.transaction,multisig_acc2)

redeem_script present
script: OP_2 03e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e4 031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee OP_2 OP_CHECKMULTISIG
sign_input_multisig commands: [b'0D\x02 3?i~\x85\x9d\xac\xf0)\xfb\xea\xc4\x13\x0b\xdb5\t\xc5\x19\xfe\xb1\xc0\xeb\xaa|7\xc3!XV\x17\x9b\x02 Tx[\xc4\xbf[\xa6f\x99\x89\xfa\x93b\x97og\x91\x98\xd3;\x15aN\x1e\xe9XL\xda\xcb\x83\x99\xef\x01', b'0C\x02 g\xd3\xd0\x9d*?P\x9d\xe7}\xf8\x07\xa8\x9c=\x96\x8e\xde*@?\x80\x19\xa8\x1cQ\xbf\xeb\x12y\x9fu\x02\x1f8\xad\xef\xfe\xa2\x1b\xe5j\xce\xf4\xe6cH\xcca0\xb8\xcb\xc8a#\x99\x03\x07\xf1G\xc2\xf4\xdek\xe4\x01', b"R!\x03\xe0\x7f\x96\xe5\xbaY\x841\xc0\xc9\x94I:J\xe9\x88\xc9\x85L\x17\x1d]K\xb1@\xdb\n'\xa4\xc8S\xe4!\x03\x1bc\xf9d\xd8\xc6]\x1d\x116\xdc\xfeP3\xde\xde\xa8\x8c-A\x194\xeaH\xc9p\x84\x10\xbe\x84\xe5\xeeR\xae"]


In [17]:
my_tx_final = MultiSigTransaction.verify_signatures(my_tx_signed, multisig_acc2)

script_pubkey of tx IN: OP_HASH160 c104b576f5436309587aefa3ddddd5c295b90480 OP_EQUAL
tx_in.script_sig.cmds[0]: 0
scriptPubKey: OP_HASH160 c104b576f5436309587aefa3ddddd5c295b90480 OP_EQUAL
commands redeemscript: b"R!\x03\xe0\x7f\x96\xe5\xbaY\x841\xc0\xc9\x94I:J\xe9\x88\xc9\x85L\x17\x1d]K\xb1@\xdb\n'\xa4\xc8S\xe4!\x03\x1bc\xf9d\xd8\xc6]\x1d\x116\xdc\xfeP3\xde\xde\xa8\x8c-A\x194\xeaH\xc9p\x84\x10\xbe\x84\xe5\xeeR\xae"
rawredeem: b"GR!\x03\xe0\x7f\x96\xe5\xbaY\x841\xc0\xc9\x94I:J\xe9\x88\xc9\x85L\x17\x1d]K\xb1@\xdb\n'\xa4\xc8S\xe4!\x03\x1bc\xf9d\xd8\xc6]\x1d\x116\xdc\xfeP3\xde\xde\xa8\x8c-A\x194\xeaH\xc9p\x84\x10\xbe\x84\xe5\xeeR\xae"
length script: 71
current_byte : 82
op_code : 82
current_byte : 33
current_byte : 33
current_byte : 82
op_code : 82
current_byte : 174
op_code : 174
redeem_script present
script: OP_2 03e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e4 031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee OP_2 OP_CHECKMULTISIG

combined script[0, b

In [20]:
my_tx_final.serialize().hex()

'0100000001cc2f5d7f42c4a3e594235b837afa7de482d455e14748c5778576773bea282c0301000000d8004730440220333f697e859dacf029fbeac4130bdb3509c519feb1c0ebaa7c37c3215856179b022054785bc4bf5ba6669989fa9362976f679198d33b15614e1ee9584cdacb8399ef01463043022067d3d09d2a3f509de77df807a89c3d968ede2a403f8019a81c51bfeb12799f75021f38adeffea21be56acef4e66348cc6130b8cbc86123990307f147c2f4de6be40147522103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e421031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee52aeffffffff02400d0300000000001976a91452903efc1004de01883ba3687be2a8ea4f6b1b1988ac607004000000000017a914c104b576f5436309587aefa3ddddd5c295b904808700000000'

In [18]:
prev_tx_id_list = ["7e466d8cce70d0a030aba8e400b71d679661db700094a946607df35ff23f68a9"]
my_tx = Build_TX.build_tx(prev_tx_id_list, [(account1.address, 30000)], segwitAccount, True, segwit = True,witness_address = True)

NameError: name 'Build_TX' is not defined

In [ ]:
my_tx.serialize().hex()

In [ ]:
my_tx

In [ ]:
multisig_acc.address

In [ ]:
prev_tx_id_list = ["f3c13c2ae497237a7a9b99c7c51a562aac6116e196d5f745469078af94efa1ea"]
my_tx = Build_TX.build_tx(prev_tx_id_list, [(segwitAccount.address, 500000)], multisig_acc, True)

In [21]:
if my_tx_final:
    print("True")

True


In [24]:
redeem_script = Script([2+80, *public_keys, 2 + 80, 174])
serialized_redeem = redeem_script.raw_serialize()

In [25]:
len(serialized_redeem)

71

In [27]:
sha256_selrialized_redeem = hashlib.sha256(serialized_redeem).digest()
len(sha256_selrialized_redeem)

32